## run this cell before starting spark cluster connection (corrects module error)

In [2]:
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [4]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', None)

### select the data

In [5]:
train_data = spark.read.json("/user/sprenner/training_april_official_hourly_prefetching_train_random15.json")
test_data = spark.read.json("/user/sprenner/training_april_official_hourly_prefetching_test_random15.json")

In [6]:
assembler_train = VectorAssembler(inputCols=["accountHash","clientStateHash","count","datasetHash","datasetScopeHash","dataset_versionHash",
                                       "datatypeHash","eventTypeHash","eventVersionHash","filenameHash", "scopeHash","filesize_double","hostnameHash",
                                       "hourHash","int_percentage","prod_stepHash","projectHash","protocolHash","remoteSiteHash","run_numberHash",
                                       "stream_nameHash","traceIdHash","traceIpHash","traceTimeentryUnix","transferStart","uuidHash"], 
                            outputCol="features")
assembler_test = VectorAssembler(inputCols=["accountHash","clientStateHash","count","datasetHash","datasetScopeHash","dataset_versionHash",
                                       "datatypeHash","eventTypeHash","eventVersionHash","filenameHash", "scopeHash","filesize_double","hostnameHash",
                                       "hourHash","int_percentage","prod_stepHash","projectHash","protocolHash","remoteSiteHash","run_numberHash",
                                       "stream_nameHash","traceIdHash","traceIpHash","traceTimeentryUnix","transferStart","uuidHash"], 
                            outputCol="features")

output_train = assembler_train.transform(train_data)
output_test = assembler_test.transform(test_data)

In [7]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

rf_percentage = RandomForestClassifier(labelCol='bucketedPercentage', featuresCol='features', predictionCol="prediction_percentage", numTrees=300, seed=1234)
model_percentage = rf_percentage.fit(output_train)
predictions_percentage = model_percentage.transform(output_test)

evaluator = MulticlassClassificationEvaluator(labelCol="bucketedPercentage", predictionCol="prediction_percentage", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_percentage)
print("Test Error Percentage = %g" % (1.0 - accuracy))
predictions_percentage = predictions_percentage.drop('rawPrediction','probability')


rf_hits = RandomForestClassifier(labelCol='bucketedHits', featuresCol='features', predictionCol="prediction_hits",numTrees=300, seed=1234)
model_hits = rf_hits.fit(output_train)
predictions_hits_percentage = model_hits.transform(predictions_percentage)

evaluator = MulticlassClassificationEvaluator(labelCol="bucketedHits", predictionCol="prediction_hits", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_hits_percentage)
print("Test Error Hits = %g" % (1.0 - accuracy))

#error percentage 80: 0.0904766
#error hits 80: 0.043354

#error percentage 50: 0.10793
#error hits 50: 0.0432863

#Test Error Percentage 30 = 0.108954
#Test Error Hits 30 = 0.0432615

#Test Error Percentage 15 = 0.0674144
#Test Error Hits 15 = 0.0432277

Test Error Percentage = 0.0674144
Test Error Hits = 0.0432277


In [8]:
model_percentage.featureImportances

SparseVector(26, {0: 0.038, 1: 0.1321, 2: 0.1085, 3: 0.0042, 4: 0.0077, 5: 0.0061, 6: 0.0702, 8: 0.0001, 9: 0.0, 10: 0.0095, 11: 0.1041, 12: 0.0274, 13: 0.0021, 14: 0.3997, 15: 0.0212, 16: 0.0096, 17: 0.0007, 18: 0.0025, 19: 0.0015, 20: 0.0047, 21: 0.0, 22: 0.0263, 23: 0.0085, 24: 0.0064, 25: 0.0089})

In [9]:
model_hits.featureImportances

SparseVector(26, {0: 0.0234, 1: 0.2808, 2: 0.0229, 3: 0.0005, 4: 0.0056, 5: 0.0036, 6: 0.037, 8: 0.0009, 9: 0.0, 10: 0.0052, 11: 0.08, 12: 0.005, 13: 0.0007, 14: 0.4384, 15: 0.0032, 16: 0.0045, 17: 0.0403, 18: 0.008, 19: 0.0007, 20: 0.0073, 21: 0.0, 22: 0.0178, 23: 0.0057, 24: 0.0062, 25: 0.0021})

In [10]:
predictions_hits_percentage = predictions_hits_percentage.withColumn("filesize", predictions_hits_percentage["filesize_double"].cast(IntegerType()))

In [11]:
final_output = predictions_hits_percentage.select('uuid', 'account', 'datasetScope', 'dataset', 'fileScope','filename', 'eventType', 'clientState', 
                                  from_unixtime(predictions_hits_percentage.traceTimeentryUnix, format='yyyy-MM-dd').alias('day'), 'traceTimeentryUnix', 
                                  'filesize', 'prediction_hits', 'int_percentage','prediction_percentage').sort(col("day").asc())

In [12]:
final_output.count()

155664

In [13]:
final_output.limit(10).toPandas()

uuid   account  datasetScope  \
0  65283411a04d4948a085b20bb7df6164    bochen    mc16_13TeV   
1  326af84d062546c99ff4e4b289b3ed69  bejaeger  data15_13TeV   
2  65283411a04d4948a085b20bb7df6164    bochen    mc16_13TeV   
3  326af84d062546c99ff4e4b289b3ed69  bejaeger  data15_13TeV   
4  326af84d062546c99ff4e4b289b3ed69  bejaeger  data15_13TeV   
5  65283411a04d4948a085b20bb7df6164    bochen    mc16_13TeV   
6  65283411a04d4948a085b20bb7df6164    bochen    mc16_13TeV   
7  a718c2cf4f9b427989a16d33b1a150b4  bchargei    mc15_14TeV   
8  65283411a04d4948a085b20bb7df6164    bochen    mc16_13TeV   
9  65283411a04d4948a085b20bb7df6164    bochen    mc16_13TeV   

                                             dataset     fileScope  \
0  mc16_13TeV.361027.Pythia8EvtGen_A14NNPDF23LO_j...    mc16_13TeV   
1  data15_13TeV.00278748.physics_Main.deriv.DAOD_...  data15_13TeV   
2  mc16_13TeV.361027.Pythia8EvtGen_A14NNPDF23LO_j...    mc16_13TeV   
3  data15_13TeV.00278748.physics_Main.deriv.DAOD_...  data15_13TeV   
4  data15_13TeV.00278748.physics_Main.deriv.DAOD_...  data15_13TeV   
5  mc16_13TeV.361028.Pythia8EvtGen_A14NNPDF23LO_j...    mc16_13TeV   
6  mc16_13TeV.361028.Pythia8EvtGen_A14NNPDF23LO_j...    mc16_13TeV   
7  mc15_14TeV.147910.Pythia8_AU2CT10_jetjet_JZ0W....    mc15_14TeV   
8  mc16_13TeV.361028.Pythia8EvtGen_A14NNPDF23LO_j...    mc16_13TeV   
9  mc16_13TeV.361027.Pythia8EvtGen_A14NNPDF23LO_j...    mc16_13TeV   

                                  filename eventType clientState         day  \
0         AOD.11185564._000439.pool.root.1  download        DONE  2018-04-01   
1  DAOD_JETM1.12601869._000013.pool.root.1  download        DONE  2018-04-01   
2         AOD.11185564._000182.pool.root.1  download        DONE  2018-04-01   
3  DAOD_JETM1.12601869._000001.pool.root.1  download        DONE  2018-04-01   
4  DAOD_JETM1.12601869._000002.pool.root.1  download        DONE  2018-04-01   
5         AOD.11183842._000424.pool.root.1  download        DONE  2018-04-01   
6         AOD.11183842._000460.pool.root.1  download        DONE  2018-04-01   
7         AOD.11442181._004555.pool.root.1  download        DONE  2018-04-01   
8         AOD.11185569._000006.pool.root.1  download        DONE  2018-04-01   
9         AOD.11185564._000867.pool.root.1  download        DONE  2018-04-01   

   traceTimeentryUnix    filesize  prediction_hits  int_percentage  \
0        1.522558e+09  2147483647              0.0               1   
1        1.522563e+09  1297191213              0.0              60   
2        1.522567e+09  2147483647              0.0               1   
3        1.522562e+09  1299492257              0.0              14   
4        1.522563e+09  1326026976              0.0              47   
5        1.522570e+09  2147483647              0.0               1   
6        1.522570e+09  2147483647              0.0               1   
7        1.522615e+09   484318334              0.0               1   
8        1.522580e+09  2147483647              0.0               1   
9        1.522559e+09  2147483647              0.0               1   

   prediction_percentage  
0                    0.0  
1                    1.0  
2                    0.0  
3                    1.0  
4                    1.0  
5                    0.0  
6                    0.0  
7                    0.0  
8                    0.0  
9                    0.0

In [14]:
final_output = final_output.orderBy("traceTimeentryUnix", ascending=True)
final_output.limit(10).toPandas()

uuid   account  datasetScope  \
0  dc3a2907f1964b75bdaaaf4cfcc61577      yliu  data15_13TeV   
1  606aa9f5a21348d2a0eeb660198d6600      yliu  data16_13TeV   
2  dc3a2907f1964b75bdaaaf4cfcc61577      yliu  data15_13TeV   
3  dc3a2907f1964b75bdaaaf4cfcc61577      yliu  data15_13TeV   
4  dc3a2907f1964b75bdaaaf4cfcc61577      yliu  data15_13TeV   
5  dc3a2907f1964b75bdaaaf4cfcc61577      yliu  data15_13TeV   
6  1edca3dd48f448c38eec86c62d6f1794  ssolomon  data17_13TeV   
7  1edca3dd48f448c38eec86c62d6f1794  ssolomon  data17_13TeV   
8  11e5cdab97b94575a5b4de8f8caee821  ssolomon  data17_13TeV   
9  dc3a2907f1964b75bdaaaf4cfcc61577      yliu  data15_13TeV   

                                             dataset     fileScope  \
0  data15_13TeV.00280231.physics_Main.deriv.DAOD_...  data15_13TeV   
1  data16_13TeV.00302053.physics_Main.deriv.DAOD_...  data16_13TeV   
2  data15_13TeV.00280231.physics_Main.deriv.DAOD_...  data15_13TeV   
3  data15_13TeV.00280231.physics_Main.deriv.DAOD_...  data15_13TeV   
4  data15_13TeV.00280231.physics_Main.deriv.DAOD_...  data15_13TeV   
5  data15_13TeV.00280231.physics_Main.deriv.DAOD_...  data15_13TeV   
6  data17_13TeV.00339070.physics_EnhancedBias.mer...  data17_13TeV   
7  data17_13TeV.00339070.physics_EnhancedBias.mer...  data17_13TeV   
8  data17_13TeV.00339070.physics_EnhancedBias.mer...  data17_13TeV   
9  data15_13TeV.00280231.physics_Main.deriv.DAOD_...  data15_13TeV   

                                    filename eventType         clientState  \
0  DAOD_HIGG5D1.11568050._000282.pool.root.1  download                DONE   
1  DAOD_HIGG5D1.11571984._001455.pool.root.1  download  ServiceUnavailable   
2  DAOD_HIGG5D1.11568050._000290.pool.root.1  download                DONE   
3  DAOD_HIGG5D1.11568050._000289.pool.root.1  download                DONE   
4  DAOD_HIGG5D1.11568050._000292.pool.root.1  download                DONE   
5  DAOD_HIGG5D1.11568050._000294.pool.root.1  download                DONE   
6   HIST_HLTMON.13559902._000001.pool.root.1  download  ServiceUnavailable   
7   HIST_HLTMON.13559902._000001.pool.root.1  download                DONE   
8   HIST_HLTMON.13559902._000001.pool.root.1  download                DONE   
9  DAOD_HIGG5D1.11568050._000297.pool.root.1  download                DONE   

          day  traceTimeentryUnix    filesize  prediction_hits  \
0  2018-04-01        1.522535e+09   416900305              0.0   
1  2018-04-01        1.522536e+09  1235302553              1.0   
2  2018-04-01        1.522536e+09   425840609              0.0   
3  2018-04-01        1.522536e+09   412359834              0.0   
4  2018-04-01        1.522538e+09   346670770              0.0   
5  2018-04-01        1.522538e+09   414758622              0.0   
6  2018-04-01        1.522538e+09     8157216              1.0   
7  2018-04-01        1.522538e+09     8157216              0.0   
8  2018-04-01        1.522538e+09     8157216              0.0   
9  2018-04-01        1.522539e+09   366031303              0.0   

   int_percentage  prediction_percentage  
0               1                    0.0  
1               0                    0.0  
2               3                    0.0  
3               3                    0.0  
4               1                    0.0  
5               1                    0.0  
6               0                    0.0  
7             100                    1.0  
8             100                    1.0  
9               3                    0.0

## get complete amount of data downloaded in time window to calculate good cache size

In [15]:
final_output.createOrReplaceTempView('output')
spark.sql("SELECT SUM(filesize) FROM (SELECT DISTINCT fileScope, fileName, filesize FROM output)").toPandas()
#54968479824024

sum(filesize)
0  55023087461878

In [16]:
final_output.write.csv('/user/sprenner/forest_april_official_hourly_prefetching15_numTrees300_seed1234_cache01.csv', sep='\t')

## predict downloads for may with trained model

In [17]:
df_mai = spark.read.json("/user/sprenner/training_official_mai_hourly_proportion_prefetching.json")

In [18]:
df_mai.describe().toPandas()

summary   account           accountHash         clientState  \
0   count    589283                589283              589283   
1    mean      None  -7.873822054121874E7                None   
2  stddev      None  1.1303640225554886E9                None   
3     min     abell           -2123889579  CannotAuthenticate   
4     max  zmarshal            2140382084      SourceNotFound   

         clientStateHash               count  \
0                 589283              578132   
1  -1.0635692705599144E9  1573.6514117883114   
2   2.7545692038495237E8    10628.4598251864   
3            -1913538301                   1   
4              634051254              195454   

                                             dataset           datasetHash  \
0                                             589283                589283   
1                                               None  -3.916131021861822E7   
2                                               None  1.2336014929229856E9   
3  data12_8TeV.00200805.physics_MinBias.recon.NTU...           -2147225360   
4  mc16_valid.361034.Pythia8EvtGen_A2MSTW2008LO_m...            2147407155   

  datasetScope      datasetScopeHash dataset_version   dataset_versionHash  \
0       589283                589283          589283                589283   
1         None  -9.356107326362851E8            None  -7.824071159882942E7   
2         None   6.784957656574693E8            None  1.1845690301436505E9   
3  data12_8TeV           -2057006107     c1128_m1842           -2147305504   
4   mc16_valid            2109319881      x563_m1972            2147255052   

  datatype          datatypeHash eventType eventTypeHash eventVersion  \
0   589283                589283    589283        589283       589283   
1     None  2.1895068833955333E8      None    5.937808E7         None   
2     None  1.0743326832603388E9      None           0.0         None   
3      AOD           -2070622869  download      59378080      1.14.10   
4      log            2124512252  download      59378080       1.16.1   

        eventVersionHash    fileScope                            filename  \
0                 589283       589283                              589283   
1  -2.0115120046498694E9         None                                None   
2    3.165646873326387E8         None                                None   
3            -2044360925  data12_8TeV    AOD.05341490._000446.pool.root.1   
4             1261702107   mc16_valid  log.14217468._003414.job.log.tgz.1   

          filenameHash       filesize_double                hits  \
0               589283                589283              589283   
1    684033.7623824207  1.4339739956710258E9  2.9973221694839323   
2  1.239027645346361E9   2.193139944581201E9  3.8806741048619764   
3          -2147471303                9576.0                   1   
4           2147478625       1.1976018886E10                  74   

                   hostname          hostnameHash           hour  \
0                    589283                589283         589283   
1                      None  -4.071422388835245E7           None   
2                      None   9.756384901185802E8           None   
3    ReCaS-ui-01.cs.infn.it           -2138306752  2018-05-01 00   
4  xenia.nevis.columbia.edu            2129595856  2018-05-31 23   

               hourHash                  id       id_proportion  \
0                589283              589283              357853   
1    -3414275.108615046            294642.0     300680.44414606   
2  1.2200577022275481E9  170111.49367694117  163494.92291812552   
3           -2140578122                   1                   2   
4            2146843273              589283              589283   

       int_percentage                                    join_key_lookup  \
0              357853                                             578132   
1  31.659882689260673                                               None   
2  31.138705245748014            

In [19]:
df_mai = df_mai.fillna(0,['count', 'int_percentage','max_percentage'])

In [20]:
from pyspark.ml.feature import Bucketizer


splits_percentage_mai = [-float("inf"), 16, float("inf")]
bucketizer_mai = Bucketizer(splits=splits_percentage_mai, inputCol="max_percentage", outputCol="bucketedPercentage")
df_mai = bucketizer_mai.transform(df_mai)

splits_hits_mai = [-float("inf"), 2, float("inf")]
bucketizer_mai = Bucketizer(splits=splits_hits_mai, inputCol="hits", outputCol="bucketedHits")
bucketedData_mai = bucketizer_mai.transform(df_mai)
bucketedData_mai.createOrReplaceTempView("bucket_mai")

#df_mai_final = bucketedData_mai.withColumn("fileScopeHash", hash(bucketedData_mai['fileScope']))
df_mai_final = bucketedData_mai.orderBy("traceTimeentryUnix", ascending=True)

In [21]:
assembler_mai = VectorAssembler(
    inputCols=["accountHash","clientStateHash","count","datasetHash","datasetScopeHash","dataset_versionHash",
                "datatypeHash","eventTypeHash","eventVersionHash","filenameHash","scopeHash","filesize_double","hostnameHash",
                "hourHash","int_percentage","prod_stepHash","projectHash","protocolHash","remoteSiteHash","run_numberHash",
                "stream_nameHash","traceIdHash","traceIpHash","traceTimeentryUnix","transferStart","uuidHash"],
    outputCol="features")

output_mai = assembler_mai.transform(df_mai_final)

In [22]:
predictions_percentage_mai = model_percentage.transform(output_mai)
evaluator = MulticlassClassificationEvaluator(labelCol="bucketedPercentage", predictionCol="prediction_percentage", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_percentage_mai)
print("Test Error Percentage Mai = %g" % (1.0 - accuracy))
predictions_percentage_mai = predictions_percentage_mai.drop('rawPrediction','probability')

predictions_hits_percentage_mai = model_hits.transform(predictions_percentage_mai)

evaluator = MulticlassClassificationEvaluator(labelCol="bucketedHits", predictionCol="prediction_hits", metricName="accuracy")
accuracy = evaluator.evaluate(predictions_hits_percentage_mai)
print("Test Error Hits Mai = %g" % (1.0 - accuracy))

#percentage mai 80: 0.126931
#hits mai 80: 0.131338

#percentage mai 50: 0.123576
#hits mai 50: 0.130177

#Test Error Percentage Mai 30 = 0.120112
#Test Error Hits Mai 30 = 0.127368

#Test Error Percentage Mai 15 = 0.163933
#Test Error Hits Mai 15 = 0.127911

Test Error Percentage Mai = 0.163933
Test Error Hits Mai = 0.127911


In [23]:
predictions_hits_percentage_mai=predictions_hits_percentage_mai.withColumn("filesize", predictions_hits_percentage_mai["filesize_double"].cast(IntegerType()))

In [24]:
predictions_hits_percentage_mai = predictions_hits_percentage_mai.drop('filesize_double')

In [25]:
final_output_mai = predictions_hits_percentage_mai.select('uuid', 'account', 'datasetScope', 'dataset', 'fileScope','filename', 'eventType', 'clientState', 
                                  from_unixtime(predictions_hits_percentage_mai.traceTimeentryUnix, format='yyyy-MM-dd').alias('day'), 'traceTimeentryUnix', 
                                  'filesize', 'prediction_hits', 'int_percentage','prediction_percentage').sort(col("day").asc())

In [26]:
final_output_mai = final_output_mai.orderBy("traceTimeentryUnix", ascending=True)

In [27]:
final_output_mai.createOrReplaceTempView('output_mai')
spark.sql("SELECT SUM(filesize) FROM (SELECT DISTINCT fileScope, fileName, filesize FROM output_mai)").toPandas()
#209595321347926

sum(filesize)
0  209595321347926

In [28]:
final_output_mai.count()

589283

In [29]:
final_output_mai = final_output_mai.orderBy("traceTimeentryUnix", ascending=True)

In [30]:
final_output_mai.write.csv('/user/sprenner/forest_mai_official_hourly_prefetching15_numTrees300_seed1234_cache01.csv', sep='\t')